# Testing Code for the Answer Model

imports

In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
# from transformers import BertTokenizer

Getting the Data/Cleaning:

In [18]:
Answer_File = '../Data/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt'

In [19]:
with open(Answer_File,'r',encoding="ISO-8859-1") as f:
    Questions = f.read().split('\n')

In [20]:
Data = {key:[] for key in Questions[0].split('\t')}

In [21]:
keys = list(Data.keys())

In [22]:
for row in range(1,len(Questions)):
    data_point = Questions[row].split('\t')
    if len(data_point)>1:
        for i in range(len(data_point)):
            Data[keys[i]].append(data_point[i])

In [23]:
df = pd.DataFrame(Data)

In [24]:
df.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,data/set4/a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,data/set4/a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,data/set4/a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,data/set4/a10


## Parsing a text file input

In [25]:
nlp = spacy.load('en_core_web_sm')

In [26]:
def is_token_allowed(token):
    '''
        Only allow valid tokens which are not stop words
        and punctuation symbols.
    '''
    if (not token or not str(token).strip() or token.is_stop or token.is_punct):
        return False
    return True

def preprocess_token(token):
    # Reduce token to its lowercase lemma form
    return token.lemma_.strip().lower()

We can now do this for every single input and use these tokens as inputs to our model

In [42]:
def Get_Tokens(file_path):
    with open('../Data/Question_Answer_Dataset_v1.2/'+'S10/'+file_path+'.txt','r') as g:
        text = g.read()
    Article = nlp(text)
    return Article
#     complete_filtered_tokens = [preprocess_token(token) for token in Article if is_token_allowed(token)]
#     return complete_filtered_tokens

In [37]:
def Get_Token_Sentences(file_path):
    with open('../Data/Question_Answer_Dataset_v1.2/'+'S10/'+file_path+'.txt','r') as g:
        text = g.read()
    Article = nlp(text)
    complete_filtered_tokens = [token for token in Article if token]
    return complete_filtered_tokens

In [ ]:
from collections import defaultdict

Articles = defaultdict(list)

for i,path in enumerate(df['ArticleFile']):
    if df['ArticleTitle'][i] not in list(Articles.keys()):
        Articles[df['ArticleTitle'][i]] = Get_Tokens(path)

# Finding Similar Sentences

In [89]:
def find_similar_sentences(raw_text,question):
    nlp = spacy.load('en_core_web_md')
    
    # Break the text into sentences
    nlp.add_pipe('sentencizer') # updated
    
    question = nlp(question)
    sentences = [sent.text.strip() for sent in raw_text.sents]
    highly_similar_sentences = {}
    for i,sentence in enumerate(sentences):
        sentence = nlp(sentence)
        sentence_no_stop_words = nlp(' '.join([str(t) for t in sentence if not t.is_stop]))
        question_no_stop_words = nlp(' '.join([str(t) for t in question if not t.is_stop]))
        
        sim = sentence_no_stop_words.similarity(question_no_stop_words)
        if sim >= 0.7:
            highly_similar_sentences.update({i:sim})
            
    return np.array(sentences)[list(highly_similar_sentences.keys())],highly_similar_sentences



In [90]:
df.iloc[0]['Question']

'Was Alessandro Volta a professor of chemistry?'

In [91]:
similar_sentences, scores = find_similar_sentences(Articles[df.iloc[0]['ArticleTitle']],df.iloc[0]['Question'])

<ipython-input-89-5aed2303a576>:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = sentence_no_stop_words.similarity(question_no_stop_words)


In [94]:
similar_sentences

array(['In 1774 he became a professor of physics at the Royal School in Como.'],
      dtype='<U384')

In [88]:
df.iloc[0]['Answer']

'Alessandro Volta was not a professor of chemistry.'